RC Circuit Design using ML

## Data Generation and preprocessing

In [ ]:
import pandas as pd
import numpy as np

r_in = np.linspace(100, 1e6, 100) 
c_in = np.linspace(1e-12, 1e-6, 100)


tolerances = [(0.01, 0.01), (0.05, 0.05), (0.10, 0.10)]  


data = []
for R_nominal in r_in:
    for C_nominal in c_in:
        for r_tol, c_tol in tolerances:
            
            R_actual = R_nominal * (1 + np.random.uniform(-r_tol, r_tol))
            C_actual = C_nominal * (1 + np.random.uniform(-c_tol, c_tol))
   

            f_nominal = 1 / (2 * np.pi * R_nominal * C_nominal)
            f_actual = 1 / (2 * np.pi * R_actual * C_actual)
            data.append([R_nominal, C_nominal, R_actual, C_actual, f_nominal, f_actual, r_tol, c_tol])

columns = ['Resistance Nominal (Ω)', 'Capacitance Nominal (F)', 'Resistance Actual (Ω)', 
           'Capacitance Actual (F)', 'Frequency Nominal (Hz)', 'Frequency Actual (Hz)', 
           'R Tolerance', 'C Tolerance']
df = pd.DataFrame(data, columns=columns)

df.to_csv("synthetic_rc_circuit_data.csv", index=False)

In [23]:
df

,Resistance Nominal (Ω),Capacitance Nominal (F),Resistance Actual (Ω),Capacitance Actual (F),Frequency Nominal (Hz),Frequency Actual (Hz),R Tolerance,C Tolerance
0,100.0,1.000000e-12,1.008676e+02,9.943160e-13,1.591549e+09,1.586880e+09,0.01,0.01
1,100.0,1.000000e-12,1.042729e+02,9.798098e-13,1.591549e+09,1.557783e+09,0.05,0.05
2,100.0,1.000000e-12,9.828577e+01,1.098514e-12,1.591549e+09,1.474089e+09,0.10,0.10
3,100.0,1.010200e-08,1.003036e+02,1.004783e-08,1.575480e+05,1.579179e+05,0.01,0.01
4,100.0,1.010200e-08,9.562006e+01,1.009890e-08,1.575480e+05,1.648152e+05,0.05,0.05
...,...,...,...,...,...,...,...,...
29995,1000000.0,9.898990e-07,9.968688e+05,9.714694e-07,1.607790e-01,1.643437e-01,0.05,0.05
29996,1000000.0,9.898990e-07,1.063398e+06,9.332766e-07,1.607790e-01,1.603666e-01,0.10,0.10
29997,1000000.0,1.000000e-06,1.009977e+06,1.000513e-06,1.591549e-01,1.575018e-01,0.01,0.01
29998,1000000.0,1.000000e-06,1.006677e+06,9.858918e-07,1.591549e-01,1.603618e-01,0.05,0.05


Data Preprocessing and Analysis

Here we perform preliminary data analysis to check if data is clean for model training

In [24]:
df = pd.read_csv("synthetic_rc_circuit_data.csv")
df

,Resistance Nominal (Ω),Capacitance Nominal (F),Resistance Actual (Ω),Capacitance Actual (F),Frequency Nominal (Hz),Frequency Actual (Hz),R Tolerance,C Tolerance
0,100.0,1.000000e-12,1.008676e+02,9.943160e-13,1.591549e+09,1.586880e+09,0.01,0.01
1,100.0,1.000000e-12,1.042729e+02,9.798098e-13,1.591549e+09,1.557783e+09,0.05,0.05
2,100.0,1.000000e-12,9.828577e+01,1.098514e-12,1.591549e+09,1.474089e+09,0.10,0.10
3,100.0,1.010200e-08,1.003036e+02,1.004783e-08,1.575480e+05,1.579179e+05,0.01,0.01
4,100.0,1.010200e-08,9.562006e+01,1.009890e-08,1.575480e+05,1.648152e+05,0.05,0.05
...,...,...,...,...,...,...,...,...
29995,1000000.0,9.898990e-07,9.968688e+05,9.714694e-07,1.607790e-01,1.643437e-01,0.05,0.05
29996,1000000.0,9.898990e-07,1.063398e+06,9.332766e-07,1.607790e-01,1.603666e-01,0.10,0.10
29997,1000000.0,1.000000e-06,1.009977e+06,1.000513e-06,1.591549e-01,1.575018e-01,0.01,0.01
29998,1000000.0,1.000000e-06,1.006677e+06,9.858918e-07,1.591549e-01,1.603618e-01,0.05,0.05


In [25]:
df.head()

,Resistance Nominal (Ω),Capacitance Nominal (F),Resistance Actual (Ω),Capacitance Actual (F),Frequency Nominal (Hz),Frequency Actual (Hz),R Tolerance,C Tolerance
0,100.0,1.000000e-12,100.867563,9.943160e-13,1.591549e+09,1.586880e+09,0.01,0.01
1,100.0,1.000000e-12,104.272904,9.798098e-13,1.591549e+09,1.557783e+09,0.05,0.05
2,100.0,1.000000e-12,98.285774,1.098514e-12,1.591549e+09,1.474089e+09,0.10,0.10
3,100.0,1.010200e-08,100.303595,1.004783e-08,1.575480e+05,1.579179e+05,0.01,0.01
4,100.0,1.010200e-08,95.620056,1.009890e-08,1.575480e+05,1.648152e+05,0.05,0.05


In [26]:
df.tail()

,Resistance Nominal (Ω),Capacitance Nominal (F),Resistance Actual (Ω),Capacitance Actual (F),Frequency Nominal (Hz),Frequency Actual (Hz),R Tolerance,C Tolerance
29995,1000000.0,9.898990e-07,9.968688e+05,9.714694e-07,0.160779,0.164344,0.05,0.05
29996,1000000.0,9.898990e-07,1.063398e+06,9.332766e-07,0.160779,0.160367,0.10,0.10
29997,1000000.0,1.000000e-06,1.009977e+06,1.000513e-06,0.159155,0.157502,0.01,0.01
29998,1000000.0,1.000000e-06,1.006677e+06,9.858918e-07,0.159155,0.160362,0.05,0.05
29999,1000000.0,1.000000e-06,1.078649e+06,1.065679e-06,0.159155,0.138457,0.10,0.10


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Resistance Nominal (Ω)   30000 non-null  float64
 1   Capacitance Nominal (F)  30000 non-null  float64
 2   Resistance Actual (Ω)    30000 non-null  float64
 3   Capacitance Actual (F)   30000 non-null  float64
 4   Frequency Nominal (Hz)   30000 non-null  float64
 5   Frequency Actual (Hz)    30000 non-null  float64
 6   R Tolerance              30000 non-null  float64
 7   C Tolerance              30000 non-null  float64
dtypes: float64(8)
memory usage: 1.8 MB


In [28]:
df.describe()

,Resistance Nominal (Ω),Capacitance Nominal (F),Resistance Actual (Ω),Capacitance Actual (F),Frequency Nominal (Hz),Frequency Actual (Hz),R Tolerance,C Tolerance
count,30000.000000,3.000000e+04,3.000000e+04,3.000000e+04,3.000000e+04,3.000000e+04,30000.000000,30000.000000
mean,500050.000000,5.000005e-07,5.000902e+05,4.998588e-07,1.673738e+05,1.622412e+05,0.053333,0.053333
std,291552.166725,2.915810e-07,2.924062e+05,2.921791e-07,1.591614e+07,1.540399e+07,0.036818,0.036818
min,100.000000,1.000000e-12,9.029883e+01,9.049554e-13,1.591549e-01,1.384566e-01,0.010000,0.010000
25%,250075.000000,2.500008e-07,2.473094e+05,2.473786e-07,4.146170e-01,4.156092e-01,0.010000,0.010000
50%,500050.000000,5.000005e-07,4.996475e+05,4.985895e-07,8.568880e-01,8.569717e-01,0.050000,0.050000
75%,750025.000000,7.500002e-07,7.521528e+05,7.509400e-07,2.420864e+00,2.432726e+00,0.100000,0.100000
max,1000000.000000,1.000000e-06,1.097413e+06,1.098554e-06,1.591549e+09,1.586880e+09,0.100000,0.100000


In [29]:
print(df.columns)

Index(['Resistance Nominal (Ω)', 'Capacitance Nominal (F)',
       'Resistance Actual (Ω)', 'Capacitance Actual (F)',
       'Frequency Nominal (Hz)', 'Frequency Actual (Hz)', 'R Tolerance',
       'C Tolerance'],
      dtype='object')


In [31]:
df.isnull()

,Resistance Nominal (Ω),Capacitance Nominal (F),Resistance Actual (Ω),Capacitance Actual (F),Frequency Nominal (Hz),Frequency Actual (Hz),R Tolerance,C Tolerance
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
29995,False,False,False,False,False,False,False,False
29996,False,False,False,False,False,False,False,False
29997,False,False,False,False,False,False,False,False
29998,False,False,False,False,False,False,False,False


In [32]:
df.isnull().sum()

Resistance Nominal (Ω)     0
Capacitance Nominal (F)    0
Resistance Actual (Ω)      0
Capacitance Actual (F)     0
Frequency Nominal (Hz)     0
Frequency Actual (Hz)      0
R Tolerance                0
C Tolerance                0
dtype: int64

## Model Training

In [33]:
# Spliting the data for training and testing

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


df = pd.read_csv("synthetic_rc_circuit_data.csv")


X = df[['Resistance Actual (Ω)', 'Capacitance Actual (F)', 'R Tolerance', 'C Tolerance']]
y = df['Frequency Actual (Hz)']


y_log = np.log1p(y)


scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_log, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

rf = RandomForestRegressor(n_estimators=200, max_depth=15, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)


y_pred_actual = np.expm1(y_pred)
y_test_actual = np.expm1(y_test)


mae = mean_absolute_error(y_test_actual, y_pred_actual)
rmse = np.sqrt(mean_squared_error(y_test_actual, y_pred_actual))
r2 = r2_score(y_test_actual, y_pred_actual)

print(f"Mean Absolute Error (MAE): {mae:.2f} Hz")
print(f"Root Mean Square Error (RMSE): {rmse:.2f} Hz")
print(f"R² Score: {r2:.4f}")

Mean Absolute Error (MAE): 467.10 Hz
Root Mean Square Error (RMSE): 12693.68 Hz
R² Score: 0.9928
